要想将选择偏向于感官输入，可以简单的使用参数化的全连接层
- 可以用一个可学习的线性变换来选择性的关注感官输入中的某些维度
- 例如你有10个感官输入的向量，你可以学一个W来控制最终输出更偏向“视觉”or“听觉”

In [1]:
import torch
from torch import nn

n_train = 50
x_train, _ = torch.sort(torch.rand(n_train) * 5)

def f(x):
    return 2 * torch.sin(x) + x ** 0.8

y_train = f(x_train) + torch.normal(0.0, 0.5, (n_train,))
x_test = torch.arange(0,5,0.1) #测试样本
y_truth = f(x_test)
n_test = len(x_test)
n_test

50

## 定义模型

In [2]:
class NWKernelRegression(nn.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = nn.Parameter(torch.randn((1,), requires_grad=True))

    def forward(self, queries, keys, values):
        # queries和attention_weights形状为查询个数
        queries = queries.repeat_interleave(keys.shape[1]).reshape((-1, keys.shape[1]))
        self.attention_weights = nn.functional.softmax(
            -((queries - keys) * self.w)**2 / 2, dim=1)
        # values的形状为(查询个数，“键－值”对个数)
        return torch.bmm(self.attention_weights.unsqueeze(1),
                         values.unsqueeze(-1)).reshape(-1)
                              

In [4]:
import math
import torch
from torch import nn
from d2l import torch as d2l